# ML Process in Databricks (Spark)

- [Machine Learning Databricks](https://docs.databricks.com/getting-started/spark/machine-learning.html)
- [Automated MLflow tracking in MLlib](https://docs.microsoft.com/en-us/azure/databricks/_static/notebooks/mllib-mlflow-integration.html)

### Load packages, methods, and data

In [0]:
# The pyspark.ml methods used in this example.
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
# https://spark.apache.org/docs/latest/ml-classification-regression.html
from pyspark.ml.regression import GBTRegressor, DecisionTreeRegressor, RandomForestRegressor, FMRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

from plotnine import *

import pyspark.sql.functions as F

import pandas as pd
# FMRegressor
# In general, in order to prevent the exploding gradient problem, it is best to scale continuous features to be between 0 and 1, or bin the continuous features and one-hot encode them.


In [0]:
# https://kb.databricks.com/machine-learning/extract-feature-info.html
def importance_plot(pipelineModel):
  va = pipelineModel.stages[-2]
  tree = pipelineModel.stages[-1]
  a = list(zip(va.getInputCols(), tree.featureImportances))
  dat_fi = pd.DataFrame({'variable': [i[0] for i in a], 'importance':[i[1] for i in a]}) \
    .sort_values('importance', ascending=False) \
    .assign(
      variable_cat = lambda x: pd.Categorical(x['variable'], categories = x.sort_values('importance', ascending=True)['variable']))

  plot = ggplot(dat_fi, aes(y = "importance", x = "variable_cat")) +\
    geom_col() +\
    coord_flip() +\
    theme_bw() +\
    labs(title = "Feature importance plot", x = "Feature")
  print(plot)
  return dat_fi

def model_results(predDF, labelCol = 'price', predictCol = 'prediction'):
  dat = predDF.select(labelCol, predictCol).toPandas()
  
  evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
  rmse = evaluator.evaluate(predDF)
  rmse = "RMSE data = %g" % rmse
  rsquared = "  R-squared:" + str(round(predDF.stat.corr('price', 'prediction'),2))

  
  p = ggplot(dat, aes(x = labelCol, y = predictCol)) +\
    geom_point() +\
    geom_abline(intercept=0, slope=1, color = "darkgrey") +\
    theme_bw() +\
    labs(title = "Predicted Vs. Actual: " +   rmse + rsquared)
  return p

### Load data from CSE 450 Housing link

In [0]:
%sh curl -O 'https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 2270k 100 2270k 0 0 9620k 0 --:--:-- --:--:-- --:--:-- 9620k

In [0]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/housing.csv,housing.csv,2324838
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/logs/,logs/,4096


In [0]:
dataset = spark.read.options(inferSchema='True').csv("file:/databricks/driver/housing.csv", header=True)
display(dataset)

id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.04799999999999,3280,4033,429900.0
3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
194000575,20141014T000000,4,1.0,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.39,1790,7488,258950.0
7522500005,20140815T000000,2,1.5,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0
3787000140,20140901T000000,3,2.25,1780,9969,1.0,0,0,3,8,1450,330,1985,0,98034,47.7286,-122.16799999999999,1950,7974,450000.0
7852110690,20140522T000000,4,2.5,2980,8107,2.0,0,0,3,9,2980,0,2000,0,98065,47.5389,-121.876,2750,7760,622500.0
5469700260,20140903T000000,4,2.25,2530,24700,2.0,0,0,3,7,2530,0,1974,0,98031,47.3939,-122.177,2650,24700,340000.0
8691410730,20150220T000000,4,2.5,3090,5600,2.0,0,0,3,9,3090,0,2005,0,98075,47.597,-121.979,3080,5788,708000.0
114100763,20140728T000000,3,0.75,1040,15000,1.0,0,0,3,6,1040,0,1941,0,98028,47.7639,-122.234,1410,19000,230000.0


### Data munging

Most of the columns seem to be useable for ML.  We need to fix the lat/long and date columns.

#### converting data sold (date)

In [0]:
dataset = dataset \
  .withColumn('date', F.to_timestamp(F.col('date').substr(1,8), 'yyyyMMdd')) \
  .withColumn('sold_year', F.year('date')) \
  .withColumn('sold_month', F.month('date'))
display(dataset.select('date', 'sold_year', 'sold_month'))

date,sold_year,sold_month
2014-11-04T00:00:00.000+0000,2014,11
2015-01-15T00:00:00.000+0000,2015,1
2014-10-14T00:00:00.000+0000,2014,10
2014-12-08T00:00:00.000+0000,2014,12
2014-08-15T00:00:00.000+0000,2014,8
2014-09-01T00:00:00.000+0000,2014,9
2014-05-22T00:00:00.000+0000,2014,5
2014-09-03T00:00:00.000+0000,2014,9
2015-02-20T00:00:00.000+0000,2015,2
2014-07-28T00:00:00.000+0000,2014,7


#### Dealing with the lat and long

I think zipcode covers the information we could glean from lat/long.  I am going to drop them. Going to try to include 4 and 3 character zip codes.

In [0]:
dataset = dataset \
  .withColumn("zip4", F.col('zipcode').substr(1,4)) \
  .withColumn("zip3", F.col('zipcode').substr(1,3))
display(dataset)

id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,sold_year,sold_month,zip4,zip3
1565930130,2014-11-04T00:00:00.000+0000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.04799999999999,3280,4033,429900.0,2014,11,9803,980
3279000420,2015-01-15T00:00:00.000+0000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,2015,1,9802,980
194000575,2014-10-14T00:00:00.000+0000,4,1.0,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,2014,10,9811,981
2115510160,2014-12-08T00:00:00.000+0000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.39,1790,7488,258950.0,2014,12,9802,980
7522500005,2014-08-15T00:00:00.000+0000,2,1.5,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,2014,8,9811,981
3787000140,2014-09-01T00:00:00.000+0000,3,2.25,1780,9969,1.0,0,0,3,8,1450,330,1985,0,98034,47.7286,-122.16799999999999,1950,7974,450000.0,2014,9,9803,980
7852110690,2014-05-22T00:00:00.000+0000,4,2.5,2980,8107,2.0,0,0,3,9,2980,0,2000,0,98065,47.5389,-121.876,2750,7760,622500.0,2014,5,9806,980
5469700260,2014-09-03T00:00:00.000+0000,4,2.25,2530,24700,2.0,0,0,3,7,2530,0,1974,0,98031,47.3939,-122.177,2650,24700,340000.0,2014,9,9803,980
8691410730,2015-02-20T00:00:00.000+0000,4,2.5,3090,5600,2.0,0,0,3,9,3090,0,2005,0,98075,47.597,-121.979,3080,5788,708000.0,2015,2,9807,980
114100763,2014-07-28T00:00:00.000+0000,3,0.75,1040,15000,1.0,0,0,3,6,1040,0,1941,0,98028,47.7639,-122.234,1410,19000,230000.0,2014,7,9802,980


In [0]:
display(dataset)

id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,sold_year,sold_month,zip4,zip3
1565930130,2014-11-04T00:00:00.000+0000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.04799999999999,3280,4033,429900.0,2014,11,9803,980
3279000420,2015-01-15T00:00:00.000+0000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,2015,1,9802,980
194000575,2014-10-14T00:00:00.000+0000,4,1.0,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,2014,10,9811,981
2115510160,2014-12-08T00:00:00.000+0000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.39,1790,7488,258950.0,2014,12,9802,980
7522500005,2014-08-15T00:00:00.000+0000,2,1.5,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,2014,8,9811,981
3787000140,2014-09-01T00:00:00.000+0000,3,2.25,1780,9969,1.0,0,0,3,8,1450,330,1985,0,98034,47.7286,-122.16799999999999,1950,7974,450000.0,2014,9,9803,980
7852110690,2014-05-22T00:00:00.000+0000,4,2.5,2980,8107,2.0,0,0,3,9,2980,0,2000,0,98065,47.5389,-121.876,2750,7760,622500.0,2014,5,9806,980
5469700260,2014-09-03T00:00:00.000+0000,4,2.25,2530,24700,2.0,0,0,3,7,2530,0,1974,0,98031,47.3939,-122.177,2650,24700,340000.0,2014,9,9803,980
8691410730,2015-02-20T00:00:00.000+0000,4,2.5,3090,5600,2.0,0,0,3,9,3090,0,2005,0,98075,47.597,-121.979,3080,5788,708000.0,2015,2,9807,980
114100763,2014-07-28T00:00:00.000+0000,3,0.75,1040,15000,1.0,0,0,3,6,1040,0,1941,0,98028,47.7639,-122.234,1410,19000,230000.0,2014,7,9802,980


### Machine Learning in Spark

#### Background: Transformers, estimators, and pipelines

Three important concepts in MLlib machine learning that are illustrated in this notebook are **Transformers**, **Estimators**, and **Pipelines**.

- **Transformer**: Takes a DataFrame as input, and returns a new DataFrame. Transformers do not learn any parameters from the data and simply apply rule-based transformations to either prepare data for model training or generate predictions using a trained MLlib model. You call a transformer with a `.transform()` method.
- **Estimator**: Learns (or "fits") parameters from your DataFrame via a `.fit()` method and returns a Model, which is a transformer.
- **Pipeline**: Combines multiple steps into a single workflow that can be easily run. Creating a machine learning model typically involves setting up many different steps and iterating over them. Pipelines help you automate this process.

For more information: [ML Pipelines](https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines)

- [StringIndexer](https://spark.apache.org/docs/2.1.0/ml-features.html#stringindexer)
- [OneHotEncoder](https://spark.apache.org/docs/2.1.0/ml-features.html#onehotencoder)
- [VectorAssembler](https://spark.apache.org/docs/2.1.0/ml-features.html#vectorassembler)
- [Classification and Regression](https://spark.apache.org/docs/2.1.0/mllib-classification-regression.html)
    - [LogisticRegression](https://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression)  or [ML linear methods](https://spark.apache.org/docs/1.5.2/ml-linear-methods.html)
- [Pipeline](https://spark.apache.org/docs/2.1.0/ml-pipeline.html#example-pipeline)
- [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)
    - [BinaryClassificationEvaluator](https://spark.apache.org/docs/2.1.0/ml-tuning.html#cross-validation) used within [CrossValidator](https://spark.apache.org/docs/2.1.0/ml-tuning.html#cross-validation)
    - [ParamGridBuilder](https://spark.apache.org/docs/2.1.0/ml-tuning.html#cross-validation)

#### Convert categorical variables to numeric

Some machine learning algorithms, such as linear and logistic regression, require numeric features. 

The following code block illustrates how to use `StringIndexer` and `OneHotEncoder` to convert categorical variables into a set of numeric variables that only take on values 0 and 1.

- `StringIndexer` converts a column of string values to a column of label indexes. For example, it might convert the values "red", "blue", and "green" to 0, 1, and 2.
- `OneHotEncoder` maps a column of category indices to a column of binary vectors, with at most one "1" in each row that indicates the category index for that row.

One-hot encoding in Spark is a two-step process. You first use the StringIndexer, followed by the OneHotEncoder. The following code block defines the StringIndexer and OneHotEncoder but does not apply it to any data yet.

For more information:
[StringIndexer](http://spark.apache.org/docs/latest/ml-features.html#stringindexer)
[OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder)

### Machine Learning Code 

1. We need to identify the categorical, numerical, and target columns
2. Then we need to build the `Transformers`
3. We put the `Transformers` into a `pipeline`
4. Fit and evaluat the model
5. CrossValidate()

In [0]:
# column names
categoricalCols = ["zipcode", "zip3", "zip4"]
spatialCols = ["lat", "lon"] # not using
numericCols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'sold_year', 'sold_month']
targetCol = ['price'] # just going to hard code 'price' into the regression arguments.
featureInputs = [c + "OHE" for c in categoricalCols] + numericCols # we are adding the one hot encoder label (OHE) to the column names because that is done with our econder transform

In [0]:
trainDF, testDF = dataset.randomSplit([0.75, 0.25], seed=42)
print(trainDF.cache().count()) # Cache because accessing training data multiple times
print(testDF.count())

15021
4979

#### Transformers

Takes a DataFrame as input, and returns a new DataFrame. Transformers do not learn any parameters from the data and simply apply rule-based transformations to either prepare data for model training or generate predictions using a trained MLlib model. You call a transformer with a `.transform()` method.

In [0]:
# stages
# The following two lines are estimators. They return functions that we will later apply to transform the dataset.
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols]) 
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols]) 
vecAssembler = VectorAssembler(inputCols=featureInputs, outputCol="features")

#### Transformer models

The first three arguments of all the ML models are listed below.  Notice that they have default values.

- `featuresCol='features'`
- `labelCol='label'`
- `predictionCol='prediction'`

In [0]:
rf = RandomForestRegressor(numTrees=10, maxDepth=5, labelCol = 'price')
dt = DecisionTreeRegressor(maxDepth=8, labelCol = 'price')
gbt = GBTRegressor(maxDepth=8, leafCol="leafId", labelCol = 'price')
fm = FMRegressor(factorSize=2, labelCol = 'price')

my_seed = 1999
rf.setSeed(my_seed)
dt.setSeed(my_seed)
gbt.setSeed(my_seed)
fm.setSeed(my_seed)

Out[11]: FMRegressor_2a1a3bc0a571

#### Pipeline

The next short example lets you see the data object that gets passed up through the `vecAssembler`. Scroll to the right to see the created columns.

In [0]:
pipeline_short = Pipeline(stages=[stringIndexer, encoder, vecAssembler])
pipelineModel_short = pipeline_short.fit(trainDF)
display(pipelineModel_short.transform(testDF))

id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,sold_year,sold_month,zip4,zip3,zipcodeIndex,zip3Index,zip4Index,zipcodeOHE,zip3OHE,zip4OHE,features
1200019,2014-05-08T00:00:00.000+0000,4,1.75,2060,26036,1.0,0,0,4,8,1160,900,1947,0,98166,47.4444,-122.351,2590,21891,647500.0,2014,5,9816,981,45.0,1.0,13.0,"Map(vectorType -> sparse, length -> 69, indices -> List(45), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 18, indices -> List(13), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(45, 83, 88, 89, 90, 91, 92, 95, 96, 97, 98, 99, 101, 102, 103, 104), values -> List(1.0, 1.0, 4.0, 1.75, 2060.0, 26036.0, 1.0, 4.0, 8.0, 1160.0, 900.0, 1947.0, 2590.0, 21891.0, 2014.0, 5.0))"
5200087,2014-07-09T00:00:00.000+0000,4,2.5,2540,5001,2.0,0,0,3,9,2540,0,2005,0,98108,47.5423,-122.302,2360,6834,487000.0,2014,7,9810,981,56.0,1.0,5.0,"Map(vectorType -> sparse, length -> 69, indices -> List(56), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 18, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(56, 75, 88, 89, 90, 91, 92, 95, 96, 97, 99, 101, 102, 103, 104), values -> List(1.0, 1.0, 4.0, 2.5, 2540.0, 5001.0, 2.0, 3.0, 9.0, 2540.0, 2005.0, 2360.0, 6834.0, 2014.0, 7.0))"
7200080,2014-11-04T00:00:00.000+0000,4,2.0,1980,10585,1.5,0,0,2,6,1980,0,1924,0,98055,47.4836,-122.214,1360,7810,239000.0,2014,11,9805,980,36.0,0.0,0.0,"Map(vectorType -> sparse, length -> 69, indices -> List(36), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 18, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(36, 69, 70, 88, 89, 90, 91, 92, 95, 96, 97, 99, 101, 102, 103, 104), values -> List(1.0, 1.0, 1.0, 4.0, 2.0, 1980.0, 10585.0, 1.5, 2.0, 6.0, 1980.0, 1924.0, 1360.0, 7810.0, 2014.0, 11.0))"
7200179,2014-10-16T00:00:00.000+0000,2,1.0,840,12750,1.0,0,0,3,6,840,0,1925,0,98055,47.483999999999995,-122.211,1480,6969,150000.0,2014,10,9805,980,36.0,0.0,0.0,"Map(vectorType -> sparse, length -> 69, indices -> List(36), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 18, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(36, 69, 70, 88, 89, 90, 91, 92, 95, 96, 97, 99, 101, 102, 103, 104), values -> List(1.0, 1.0, 1.0, 2.0, 1.0, 840.0, 12750.0, 1.0, 3.0, 6.0, 840.0, 1925.0, 1480.0, 6969.0, 2014.0, 10.0))"
7600065,2014-06-05T00:00:00.000+0000,3,2.25,1530,1245,2.0,0,0,3,9,1050,480,2014,0,98122,47.6018,-122.29700000000001,1530,2307,465000.0,2014,6,9812,981,42.0,1.0,8.0,"Map(vectorType -> sparse, length -> 69, indices -> List(42), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 18, indices -> List(8), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(42, 78, 88, 89, 90, 91, 92, 95, 96, 97, 98, 99, 101, 102, 103, 104), values -> List(1.0, 1.0, 3.0, 2.25, 1530.0, 1245.0, 2.0, 3.0, 9.0, 1050.0, 480.0, 2014.0, 1530.0, 2307.0, 2014.0, 6.0))"
11200400,2014-09-23T00:00:00.000+0000,3,2.5,1910,4488,2.0,0,0,3,8,1910,0,1998,0,98007,47.6176,-122.14,1530,3696,617000.0,2014,9,9800,980,59.0,0.0,3.0,"Map(vectorType -> sparse, length -> 69, indices -> List(59), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 18, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 105, indices -> List(59, 69, 73, 88, 89

#### Model Pipelines

I want to look at two different models.

##### Gradient Boosted Tree

In [0]:
pipeline_gbt = Pipeline(stages=[stringIndexer, encoder, vecAssembler, gbt]) # Define the pipeline based on the stages created in previous steps that works all the way to the model.
Model_gbt = pipeline_gbt.fit(trainDF) # Define the pipeline model.
predDF_gbt = Model_gbt.transform(testDF) # Apply the pipeline model to the test dataset.

In [0]:
print(model_results(predDF_gbt))
importance_plot(Model_gbt)


Out[14]:

,variable,importance,variable_cat
14,yr_built,0.020262,yr_built
7,floors,0.016300,floors
5,sqft_living,0.013348,sqft_living
8,waterfront,0.009937,waterfront
19,sold_month,0.009931,sold_month
3,bedrooms,0.008970,bedrooms
12,sqft_above,0.008657,sqft_above
0,zipcodeOHE,0.007988,zipcodeOHE
6,sqft_lot,0.005501,sqft_lot
9,view,0.002101,view


d
 ##### Decision Tree

In [0]:
pipeline_dt = Pipeline(stages=[stringIndexer, encoder, vecAssembler, dt]) # Define the pipeline based on the stages created in previous steps that works all the way to the model.
Model_dt = pipeline_dt.fit(trainDF) # Define the pipeline model.
predDF_dt = Model_dt.transform(testDF) # Apply the pipeline model to the test dataset.

print(model_results(predDF_dt))
importance_plot(Model_dt)

Out[15]:

,variable,importance,variable_cat
19,sold_month,0.010323,sold_month
9,view,0.003377,view
7,floors,0.003246,floors
14,yr_built,0.003114,yr_built
8,waterfront,0.002720,waterfront
15,yr_renovated,0.000931,yr_renovated
17,sqft_lot15,0.000797,sqft_lot15
16,sqft_living15,0.000609,sqft_living15
6,sqft_lot,0.000344,sqft_lot
12,sqft_above,0.000000,sqft_above


##### Random Forest

In [0]:
pipeline_rf = Pipeline(stages=[stringIndexer, encoder, vecAssembler, rf]) # Define the pipeline based on the stages created in previous steps that works all the way to the model.
Model_rf = pipeline_rf.fit(trainDF) # Define the pipeline model.
predDF_rf = Model_rf.transform(testDF) # Apply the pipeline model to the test dataset.

# va = Model_rf.stages[-2]
# tree = Model_rf.stages[-1]
# print(tree.featureImportances)
# a = list(zip(va.getInputCols(), tree.featureImportances))
# print(a)


print(model_results(predDF_rf))
importance_plot(Model_rf)

Out[16]:

,variable,importance,variable_cat
9,view,0.000617,view
8,waterfront,0.000558,waterfront
14,yr_built,0.000122,yr_built
0,zipcodeOHE,0.000000,zipcodeOHE
11,grade,0.000000,grade
18,sold_year,0.000000,sold_year
17,sqft_lot15,0.000000,sqft_lot15
16,sqft_living15,0.000000,sqft_living15
15,yr_renovated,0.000000,yr_renovated
13,sqft_basement,0.000000,sqft_basement


#### Parameter tuning

Now if we want to get a better fitting model by searching the parameter space to 'tune the model'

- [ML Tuning: model selection and hyperparameter tuning](https://spark.apache.org/docs/latest/ml-tuning.html#ml-tuning-model-selection-and-hyperparameter-tuning)
- [Understanding the parallelism argument](https://bryancutler.github.io/cv-parallel/)
- __Understanding the numFolds argument:__ `CrossValidator` begins by splitting the dataset into a set of folds which are used as separate training and test datasets. E.g., with k=3 folds, CrossValidator will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing.

_Note that cross-validation over a grid of parameters is expensive._

###### Notes on GBT Regressor

From [ref 1](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GBTRegressor.html) and [ref 2](https://seahorse.deepsense.ai/operations/gbt_regression.html) the following table was constructed.

| Name                     | Type                   | Description                                                  |
| :----------------------- | :--------------------- | :----------------------------------------------------------- |
| regression impurity (`impurity`)    | `SingleChoice`         | The criterion used for information gain calculation. Possible values: `["variance"]` |
| loss function (`LossType`)          | `SingleChoice`         | The loss function which GBT tries to minimize. Possible values: `["squared", "absolute"]` |
| max bins (`maxBins=32`)              | `Numeric`              | The maximum number of bins used for discretizing continuous features and for choosing how to split on features at each node. More bins give higher granularity. Must be >= 2 and >= number of categories in any categorical feature. |
| max depth (`maxDepth=5`)              | `Numeric`              | The maximum depth of the tree. E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. |
| max iterations (`maxIter=20`)         | `Numeric`              | The maximum number of iterations.                            |
| min information gain (`minInfoGain=0`)   | `Numeric`              | The minimum information gain for a split to be considered at a tree node. |
| min instances per node (`minInstancesPerNode=1`) | `Numeric`              | The minimum number of instances each child must have after split. If a split causes the left or right child to have fewer instances than the parameter's value, the split will be discarded as invalid. |
| seed (`seed=None`)                   | `Numeric`              | The random seed.                                             |
| step size (`stepSize=0.1`)              | `Numeric`              | The step size to be used for each iteration of optimization. |
| subsampling rate (`subsamplingRate=1`)       | `Numeric`              | The fraction of the training data used for learning each decision tree. |
| label column (`labelCol='label'`)           | `SingleColumnSelector` | The label column for model fitting.                          |
| features column (`featuresCol='features'`)        | `SingleColumnSelector` | The features column for model fitting.                       |
| prediction column (`predictionCol='prediction'`)      | `String`               | The prediction column created during model scoring.          |

##### Cross validation for GBT (takes a while)

In [0]:

paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [5, 8, 10])
             .addGrid(gbt.minInstancesPerNode, [1, 3, 10])
             .addGrid(gbt.maxIter, [20, 50])
             .addGrid(gbt.maxBins, [5, 15, 25, 32])
             .build())

regEvaluator = RegressionEvaluator(metricName="rmse", labelCol='price')

cv = CrossValidator(estimator=pipeline_gbt, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=regEvaluator, 
                    numFolds=3, # Create a 3-fold CrossValidator
                    parallelism = 4) 

 
# Run cross validations. This step takes a some time and returns the best model found from the cross validation. Like forever on 4 processors.
cvModel = cv.fit(trainDF)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

To view the MLflow experiment associated with the notebook, click the **Experiment** icon in the notebook context bar on the upper right. All notebook runs appear in the sidebar. To more easily compare their results, click the icon at the far right of **Experiment Runs** (it shows "View Experiment UI" when you hover over it). The Experiment page appears.

For example, to examine the effect of tuning `maxDepth`:

1. On the Experiment page, enter `params.regParam = "0.5"` in the **Search Runs** box, and click **Search**.
2. Select the resulting runs and click **Compare**.
3. In the Scatter Plot, you can select different variables to show.

In [0]:
# Use the model identified by the cross-validation to make predictions on the test dataset
# After identifying the best ParamMap, CrossValidator finally re-fits the Estimator using the best ParamMap and the entire dataset.
cvPredDF = cvModel.transform(testDF)

##### Optimal parameters

In [0]:
best_gbt = cvModel.bestModel
print("default")
display(pd.json_normalize({param[0].name: param[1] for param in gbt.extractParamMap().items()}))
print("optimal")
display(pd.json_normalize({param[0].name: param[1] for param in best_gbt.stages[-1].extractParamMap().items()}))

default

seed,maxDepth,maxBins,minInstancesPerNode,minInfoGain,maxMemoryInMB,cacheNodeIds,subsamplingRate,checkpointInterval,lossType,maxIter,stepSize,impurity,featureSubsetStrategy,validationTol,leafCol,minWeightFractionPerNode,predictionCol,featuresCol,labelCol
1999,8,32,1,0.0,256,false,1.0,10,squared,20,0.1,variance,all,0.01,leafId,0.0,prediction,features,price


optimal

cacheNodeIds,checkpointInterval,featureSubsetStrategy,featuresCol,impurity,labelCol,leafCol,lossType,maxBins,maxDepth,maxIter,maxMemoryInMB,minInfoGain,minInstancesPerNode,minWeightFractionPerNode,predictionCol,seed,stepSize,subsamplingRate,validationTol
false,10,all,features,variance,price,leafId,squared,15,8,50,256,0.0,10,0.0,prediction,1999,0.1,1.0,0.01


In [0]:
print(model_results(cvPredDF))
importance_plot(best_gbt)

Out[31]:

,variable,importance,variable_cat
14,yr_built,0.009060,yr_built
3,bedrooms,0.008060,bedrooms
8,waterfront,0.007336,waterfront
5,sqft_living,0.007283,sqft_living
12,sqft_above,0.005683,sqft_above
9,view,0.005442,view
0,zipcodeOHE,0.005318,zipcodeOHE
19,sold_month,0.004928,sold_month
7,floors,0.003223,floors
6,sqft_lot,0.001567,sqft_lot


In [0]:
regEvaluator.evaluate(cvPredDF)

Out[33]: 155882.5505092308

In [0]:
regEvaluator.explainParams()

In [0]:
# https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html
# https://kb.databricks.com/machine-learning/extract-feature-info.html
# https://runawayhorse001.github.io/LearningApacheSpark/index.html
#####
# https://docs.databricks.com/applications/machine-learning/model-export/mleap-model-export.html#export-and-import-models-in-python
# https://databricks.com/blog/2018/09/21/how-to-use-mlflow-to-reproduce-results-and-retrain-saved-keras-ml-models.html

In [0]:
import mlflow.keras
#your Keras built, trained, and tested model
model = ...
#local or remote S3 or Azure Blob path
model_dir_path=...
# save the mode to local or remote accessible path on the S3 or Azure Blob
mlflow.keras.save_model(model, model_dir_path)